docentes
autoavaliação docentes 2017.1
turmas 2017.1
matrículas em componentes em 2017.1
componentes curriculares presenciais

    docente -> comparar : avalição prof | taxa de reprovaçao
                          avaliação aluno | taxa de reprovaçao


In [1]:
# importando as bibliotecas necessárias
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [23]:
# Turmas 2017.1
# http://dados.ufrn.br/dataset/turmas
# turmas
turmas_url = 'https://bit.ly/2wn1AFI'

# matrículas em componentes em 2017.1
notas_url = 'https://bit.ly/2KMoTvS'

# componentes curriculares presenciais
disciplinas_url = 'http://bit.do/componentes_curriculares_ufrn'

# docentes da ufrn atualizado em 2018
docentes_url = 'http://bit.do/docentes_ufrn_2018'

#autoavaliação docentes
nota_docente_url = 'https://bit.ly/2IqonoW'

# turmas 2017.1
turmas = pd.read_csv(turmas_url, sep=';')

# matriculas em componentes em 2017.1
notas = pd.read_csv(notas_url, sep=';')

# componentes curriculares presenciais
# por alguma razão ou outra, esse dataset tinha algum lixo em linhas que ocasionava 
# erros de leitura, dessa forma foi realizado um filtro apenas por colunas mais importantes
disciplinas = pd.read_csv(disciplinas_url, 
                          sep=';', 
                          usecols=["id_componente","codigo","nivel","nome","unidade_responsavel"])

#filtrando notas dos docentes
docentes = pd.read_csv(docentes_url, sep=';')
nota_docente = pd.read_csv(nota_docente_url, sep=';', usecols=["id_docente", "nome_docente", "id_turma", "ano", "autoavaliacao_aluno_media", "atuacao_profissional_media", "postura_profissional_media" ])


In [24]:
nota_docente.head()

,id_docente,nome_docente,id_turma,ano,postura_profissional_media,atuacao_profissional_media,autoavaliacao_aluno_media
0,15370,FERNANDO MOREIRA DA SILVA,1209923,2013,9.17,8.13,7.61
1,15370,FERNANDO MOREIRA DA SILVA,1222008,2014,9.44,7.17,8.71
2,15370,FERNANDO MOREIRA DA SILVA,1222218,2014,9.19,6.77,8.12
3,15370,FERNANDO MOREIRA DA SILVA,1229520,2014,9.06,7.68,7.62
4,15370,FERNANDO MOREIRA DA SILVA,1229545,2014,9.27,8.45,8.48


In [12]:
# Adaptando o dataframe turma e disciplinas para o merge
# Esse procedimento é necessário pois o dataframe turma está codificado com ids

# A coluna id_componente precisa ser a mesma em ambos os dataframes
turmas.rename(columns={'id_componente_curricular': 'id_componente'}, inplace=True)

# id_componente em ambos os dataframes precisam ser do mesmo tipo
turmas.id_componente = turmas.id_componente.astype(str)

# Limpando o dataframe disciplinas com linhas desnecessárias
disciplinas.dropna(inplace=True)
disciplinas = disciplinas[~disciplinas.id_componente.duplicated()]

In [13]:
# Fazendo um merge entre o dataframe turmas e disciplinas utilizando como base a coluna id_componente
# https://pandas.pydata.org/pandas-docs/stable/merging.html
colunas = ["id_turma","id_docente_interno","codigo","nivel_ensino","nome","unidade_responsavel",
          "situacao_turma","capacidade_aluno","descricao_horario"]

# merge os dois dataframes utilizando como base a coluna id_componenteb
merge_turmas_disciplinas = pd.merge(turmas, disciplinas, on='id_componente')[colunas]

merge_turmas_disciplinas.dropna(inplace=True)
merge_turmas_disciplinas.head()

,id_turma,id_docente_interno,codigo,nivel_ensino,nome,unidade_responsavel,situacao_turma,capacidade_aluno,descricao_horario
0,57602957,5753061.0,ART2077,LATO SENSU,PRODUÇÃO COREOGRÁFICA I,DEPARTAMENTO DE ARTES,CONSOLIDADA,40.0,7M2345 7T23456
2,57602962,5752714.0,ART2083,LATO SENSU,VIESES ARTÍSTICOS CONTEMPORÂNEOS EM DANÇA ...,DEPARTAMENTO DE ARTES,ABERTA,40.0,1M345 7M2345 17T23456
3,57602963,3031948.0,ART2086,LATO SENSU,PRÁTICAS DE IMPROVISAÇÃO PARA A CENA DAS DANÇA...,DEPARTAMENTO DE ARTES,CONSOLIDADA,40.0,7M2345 7T3456
4,57603192,5752917.0,MUT491,TÉCNICO,ARRANJOS II,ESCOLA DE MÚSICA,CONSOLIDADA,20.0,3T34
5,57603193,35466.0,MUT102,TÉCNICO,CANTO II,ESCOLA DE MÚSICA,CONSOLIDADA,3.0,3T456


In [14]:
# filtrando as turmas com código IMD
turmas = merge_turmas_disciplinas[merge_turmas_disciplinas.codigo.str.contains("IMD")]

# utilizar apenas disciplinas com código "CONSOLIDADA"
turmas = turmas[turmas.situacao_turma == 'CONSOLIDADA']

# Adaptar o nome da coluna id_docente_interno para id_servidor
turmas.rename(columns={'id_docente_interno': 'id_servidor'}, inplace=True)

turmas.capacidade_aluno = turmas.capacidade_aluno.astype(np.int64)

# Adaptar os tipos de dados em turmas para o merge com o dataframe docentes
turmas.id_servidor = turmas.id_servidor.astype(np.int64)


# merge os dataframes  turmas_imd e docentes utilizando como base a coluna id_servidor

colunas = ["id_turma", "codigo", "nivel_ensino", "nome_y", "codigo", "nivel_ensino",
          "nome_x","capacidade_aluno", "descricao_horario"]

turmas_nomesprof = pd.merge(turmas, docentes, on='id_servidor')[colunas]
turmas_nomesprof.head()


,id_turma,codigo,nivel_ensino,nome_y,codigo,nivel_ensino,nome_x,capacidade_aluno,descricao_horario
0,57608376,IMD0017,GRADUAÇÃO,LUCELIO DANTAS DE AQUINO,IMD0017,GRADUAÇÃO,PRÁTICAS DE LEITURA E ESCRITA EM PORTUGUÊS I,60,5N34 (24/07/2017 - 16/12/2017)
1,57608398,IMD0027,GRADUAÇÃO,LUCELIO DANTAS DE AQUINO,IMD0027,GRADUAÇÃO,PRÁTICAS DE LEITURA E ESCRITA EM PORTUGUÊS II,60,3N34 (24/07/2017 - 16/12/2017)
2,57608691,IMD0335,GRADUAÇÃO,LUCELIO DANTAS DE AQUINO,IMD0335,GRADUAÇÃO,TÓPICOS ESPECIAIS I,40,35T56 (24/07/2017 - 16/12/2017)
3,57608377,IMD0019,GRADUAÇÃO,LOURENA KARIN DE MEDEIROS ROCHA,IMD0019,GRADUAÇÃO,RESOLUÇÃO DE PROBLEMAS MATEMÁTICOS PARA TI,65,246M3456 (24/07/2017 - 16/12/2017)
4,57608377,IMD0019,GRADUAÇÃO,CHARLES ANDRYE GALVAO MADEIRA,IMD0019,GRADUAÇÃO,RESOLUÇÃO DE PROBLEMAS MATEMÁTICOS PARA TI,65,246M3456 (24/07/2017 - 16/12/2017)


In [20]:
# filtrar apenas as turmas do IMD
notas_imd = notas[notas.id_turma.isin(turmas_nomesprof.id_turma)]

# merge entre os dataframes notas_imd e turmas_imd_nomesprof utilizando como referencia id_turma
notas_imd_merge = pd.merge(notas_imd, turmas_nomesprof, on='id_turma')
notas_imd_merge.descricao.unique()

array(['CANCELADO', 'APROVADO POR NOTA', 'APROVADO', 'REPROVADO',
       'INDEFERIDO', 'DESISTENCIA', 'REPROVADO POR MÉDIA E POR FALTAS',
       'TRANCADO', 'REPROVADO POR NOTA E FALTA', 'REPROVADO POR NOTA',
       'EXCLUIDA', 'MATRICULADO', 'REPROVADO POR FALTAS'], dtype=object)